# Pruebas de segmentación con el Catálogo Morfológico de Nebulosas Planetarias del IAC
En este documento vamos a probar las técnicas comentadas por Diego Cantorna en el documento de astrogestem (disponible en la carpeta astrosegstem de este mismo repositorio).

## 0. Preparación del entorno
Para empezar, lo primero que vamos a hacer va a ser preparar el entorno en el que se van a hacer las ejecuciones, de donde el programa tiene que cargar las imágenes y las máscaras de las nebulosas planetarias y donde va a guardar los resultados obtenidos.

In [5]:
import os

print("Vamos a cambiar el directorio de trabajo")

# Indicamos la ruta del directorio de trabajo
route = "C:\\Users\\Lucan\\OneDrive - Universidade da Coruña\\Escritorio\\4_GCEID\\TFG\\test\\PNe_segmentation"
os.chdir(route)

current_directory = os.getcwd()
print("\nEl directorio actual es:", current_directory)

# Listamos el contenido del directorio
files = os.listdir(current_directory)
print("\nContenido del directorio actual:")
for file in files:
    print("\t",file)


Vamos a cambiar el directorio de trabajo

El directorio actual es: C:\Users\Lucan\OneDrive - Universidade da Coruña\Escritorio\4_GCEID\TFG\test\PNe_segmentation

Contenido del directorio actual:
	 data
	 masks
	 segmentation_test.ipynb


## 1. Carga de las imágenes junto a sus máscaras


Vamos a hacer una prueba de como cargaríamos una máscara y contorno junto con la imagen a la que pertenece

In [12]:
# Listamos el contenido del directorio de las máscaras
masks_directory = "C:\\Users\\Lucan\\OneDrive - Universidade da Coruña\\Escritorio\\4_GCEID\\TFG\\test\\PNe_segmentation\\masks"
masksFiles = os.listdir(masks_directory)
masks_files = [file for file in masksFiles if file.endswith(".png")]

data_directory = "C:\\Users\\Lucan\\OneDrive - Universidade da Coruña\\Escritorio\\4_GCEID\\TFG\\test\\PNe_segmentation\\data"
dataFiles = os.listdir(data_directory)
data_files = [file for file in dataFiles if file.endswith(".fits")]

In [14]:
nebula_dict = {}

for mask_file in masks_files:
    nebula_name = mask_file.split("_")[0]
    nebula_dict[nebula_name] = mask_file

nebula_dict


{'A13': 'A13_contour.png',
 'A14': 'A14_mask.png',
 'A16': 'A16_contour.png',
 'A18': 'A18_mask.png',
 'A19': 'A19_mask.png',
 'A24': 'A24_contour.png',
 'A2': 'A2_mask.png',
 'A30': 'A30_mask.png',
 'A39': 'A39_mask.png',
 'A3': 'A3_contour.png',
 'A43': 'A43_mask.png',
 'A46': 'A46_mask.png',
 'A4': 'A4_contour.png',
 'A50': 'A50_mask.png',
 'A52': 'A52_mask.png',
 'A54': 'A54_mask.png',
 'A57': 'A57_mask.png',
 'A58': 'A58_mask.png',
 'A63': 'A63_mask.png',
 'A68': 'A68_mask.png',
 'A70': 'A70_mask.png',
 'A71': 'A71_mask.png',
 'A72': 'A72_mask.png',
 'A73': 'A73_contour.png',
 'A75': 'A75_mask.png',
 'A77': 'A77_mask.png',
 'A78': 'A78_mask.png',
 'A80': 'A80_mask.png',
 'A83': 'A83_contour.png',
 'A84': 'A84_mask.png',
 'A86': 'A86_contour.png',
 'A8': 'A8_mask.png',
 'Anom20H02m': 'Anom20H02m_mask.png',
 'BV5': 'BV5_3_contour.png',
 'CRL618': 'CRL618_contour.png',
 'Dd1': 'Dd1_mask.png',
 'EGB1': 'EGB1_mask.png',
 'h2': 'h2_mask.png',
 'h3': 'h3_29_contour.png',
 'HaTr10': 'HaTr

In [16]:
nebula_files_dict = {key: value for key, value in nebula_dict.items() if key in data_files}
nebula_files_dict


{}